In this notebook, a modular implementation of preprocessing steps for Sentinel-1A Dual Polarization Ground Range Detected (GRD) High Resolution (HR) data is performed.


# **Dataset Details**

    Name: Sentinel-1 Dataset
    Provider: European Space Agency (ESA)
    Data Range: 2015-2022
    Data Type: Synthetic Aperture Radar (SAR) imagery
    Purpose: Flood detection analysis and development of a machine learning model for floodwater segmentation and early warning system.
    Access: The dataset can be accessed through the ESA's Sentinels Scientific Data Hub.

# **Steps are discusssed below in brief:**

    1. Radiometric Calibration: Sentinel-1A data is provided in Digital Numbers (DN), which needs to be converted to calibrated power values in decibels (dB). This is done using the calibration parameters provided by the European Space Agency (ESA).

    2. Multilooking: The raw data is in single-look format, which means that each pixel represents the radar response from a single area on the ground. Multilooking is a technique to combine multiple adjacent pixels to reduce the noise and improve the spatial resolution. It also helps to reduce speckle, which is a multiplicative noise in radar images.

    3. Speckle Filtering: Speckle filtering is done to reduce the noise introduced by the radar signal. There are several speckle filtering techniques available, such as Lee filter, Gamma filter, and Median filter.

    4. Geocoding: Geocoding is the process of converting the radar coordinates to geographic coordinates. This is done by applying a mathematical transformation to the data using the metadata provided by ESA.

    5. Subsetting: Subsetting is the process of selecting a region of interest from the entire scene. This is done to reduce the data size and processing time.

    6. Terrain Correction: Terrain correction is a process of correcting for the topographic effects on the radar signal. This is done to remove the distortions caused by the varying elevation of the ground surface. The corrected data is referred to as the backscatter coefficient, which represents the amount of radar energy that is scattered back to the sensor from the ground.

Note: The above steps may vary depending on the specific application and requirements. 

For flood monitoring, it may be necessary to skip the **terrain correction** step as the elevation changes caused by floods may also be of interest. 

Further, **Geocoding, subsetting, and multilooking** are preprocessing steps that are typically performed on SAR data to correct for geometric distortions, reduce noise, and enhance the resolution of the images. However, in the context of flood mapping, these steps may not be necessary or may not provide significant benefits.

In [ ]:
#installing requirements
!pip install rasterio
!pip install earthpy

In [ ]:
#importing libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal
import rasterio as rio
from skimage.filters import median
import earthpy.plot as ep
import glob
from affine import Affine

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Reads in SENTINEL-1A_DUAL_POL_GRD_HIGH_RES data from the given path.
#Returns a tuple of two numpy arrays representing VV and VH polarization images.

def read_data(path, vv_file_name, vh_file_name):
    # Locate the VV and VH .tif files
    vv_path = os.path.join(path, vv_file_name)
    vh_path = os.path.join(path, vh_file_name)
    
    # Open the VV and VH images using gdal
    vv_dataset = gdal.Open(vv_path)
    vh_dataset = gdal.Open(vh_path)
    
    # Get the size of the image
    rows, cols = vv_dataset.RasterYSize, vv_dataset.RasterXSize
    
    # Define block size for reading in data
    block_size = 512
    
    # Create empty lists to store VV and VH blocks
    vv_blocks, vh_blocks = [], []
    
    # Read in VV and VH data in blocks
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            vv_block = vv_dataset.ReadAsArray(j, i, block_size, block_size)
            vh_block = vh_dataset.ReadAsArray(j, i, block_size, block_size)
            
            if vv_block is not None and vh_block is not None:
                vv_blocks.append(vv_block)
                vh_blocks.append(vh_block)
    
    # Concatenate VV and VH blocks to create final arrays
    vv = np.concatenate(vv_blocks, axis=0)
    vh = np.concatenate(vh_blocks, axis=0)
    
    # Close the datasets
    vv_dataset = None
    vh_dataset = None
    
    return vv, vh

In [ ]:
# Applies radiometric calibration to the VV and VH polarization images,
# block by block, using the calibration factors provided by the sensor.

def apply_calibration(vv, vh, block_size=512):
    # Calibration factors for VV and VH images
    k1_vv = -84.02
    k2_vv = 0.62
    k1_vh = -81.03
    k2_vh = 0.64
    
    # Get input array shape
    rows, cols = vv.shape
    
    # Create output arrays
    calibrated_vv = np.zeros((rows, cols), dtype=np.uint16)
    calibrated_vh = np.zeros((rows, cols), dtype=np.uint16)
    
    # Apply calibration to each block of the input arrays
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            vv_block = vv[i:i+block_size, j:j+block_size]
            vh_block = vh[i:i+block_size, j:j+block_size]
            
            # Apply calibration to block
            vv_block = (vv_block * k2_vv + k1_vv).astype(np.uint16)
            vh_block = (vh_block * k2_vh + k1_vh).astype(np.uint16)
            
            # Save calibrated block to output arrays
            calibrated_vv[i:i+block_size, j:j+block_size] = vv_block
            calibrated_vh[i:i+block_size, j:j+block_size] = vh_block
    
    return calibrated_vv, calibrated_vh

In [ ]:
#optimised-version

#Applies radiometric calibration to the VV and VH polarization images,
#block by block, using the calibration factors provided by the sensor.

def apply_calibration2(path, vv_file_name, vh_file_name, block_size=512):
    # Calibration factors for VV and VH images
    k1_vv = -84.02
    k2_vv = 0.62
    k1_vh = -81.03
    k2_vh = 0.64
    
    # Open the VV and VH images using gdal
    vv_path = os.path.join(path, vv_file_name)
    vh_path = os.path.join(path, vh_file_name)
    vv_dataset = gdal.Open(vv_path)
    vh_dataset = gdal.Open(vh_path)
    
    # Get the size of the image
    rows, cols = vv_dataset.RasterYSize, vv_dataset.RasterXSize
    
    # Create output arrays
    calibrated_vv = np.zeros((rows, cols), dtype=np.uint16)
    calibrated_vh = np.zeros((rows, cols), dtype=np.uint16)
    
    # Apply calibration to each block of the input arrays
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            # Read in a block of data
            vv_block = vv_dataset.ReadAsArray(j, i, block_size, block_size)
            vh_block = vh_dataset.ReadAsArray(j, i, block_size, block_size)
            
            if vv_block is not None and vh_block is not None:
                # Apply calibration to block
                vv_block = (vv_block * k2_vv + k1_vv).astype(np.uint16)
                vh_block = (vh_block * k2_vh + k1_vh).astype(np.uint16)
                
                # Save calibrated block to output arrays
                calibrated_vv[i:i+block_size, j:j+block_size] = vv_block
                calibrated_vh[i:i+block_size, j:j+block_size] = vh_block
    
    # Close the datasets
    vv_dataset = None
    vh_dataset = None
    
    # Return calibrated VV and VH images as numpy arrays
    return calibrated_vv, calibrated_vh

In [ ]:
#Applies terrain correction to the VV and VH polarization images.
#Returns the terrain-corrected VV and VH images as numpy arrays.


def apply_terrain_correction(vv, vh):
    # Conversion factor from decibels to power
    db_to_power = lambda x: 10 ** (x / 10.0)
    
    # Terrain height for the given region
    terrain_height = 0
    
    # Apply terrain correction to VV image
    vv_tc = db_to_power(vv) / np.sin(np.radians(terrain_height))
    vv_db = 10 * np.log10(vv_tc)
    
    # Apply terrain correction to VH image
    vh_tc = db_to_power(vh) / np.sin(np.radians(terrain_height))
    vh_db = 10 * np.log10(vh_tc)
    
    return vv_db, vh_db

In [ ]:
#optimised version

#Applies terrain correction to the VV and VH polarization images.
#Returns the terrain-corrected VV and VH images as numpy arrays.

def apply_terrain_correction2(vv, vh, block_size=512):
    # Conversion factor from decibels to power
    db_to_power = lambda x: 10 ** (x / 10.0)
    
    # Terrain height for the given region
    terrain_height = 0
    
    # Get input array shape
    rows, cols = vv.shape
    
    # Create output arrays
    vv_db = np.zeros((rows, cols), dtype=np.float32)
    vh_db = np.zeros((rows, cols), dtype=np.float32)
    
    # Apply terrain correction to each block of the input arrays
    for i in range(0, rows, block_size):
        for j in range(0, cols, block_size):
            vv_block = vv[i:i+block_size, j:j+block_size]
            vh_block = vh[i:i+block_size, j:j+block_size]
            
            # Apply terrain correction to block
            vv_tc = db_to_power(vv_block) / np.sin(np.radians(terrain_height))
            vh_tc = db_to_power(vh_block) / np.sin(np.radians(terrain_height))
            
            # Convert back to dB
            vv_block_db = 10 * np.log10(vv_tc)
            vh_block_db = 10 * np.log10(vh_tc)
            
            # Save terrain-corrected block to output arrays
            vv_db[i:i+block_size, j:j+block_size] = vv_block_db
            vh_db[i:i+block_size, j:j+block_size] = vh_block_db
    
    return vv_db, vh_db

In [ ]:
#Applies a speckle filter to the VV and VH polarization images.
#Returns the filtered VV and VH images as numpy arrays.

def apply_speckle_filter(vv, vh):
    vv_filtered = median(vv)
    vh_filtered = median(vh)
    
    return vv_filtered, vh_filtered


In [ ]:
#Saves the VV and VH polarization images as .tif files to the given path.

def save_images(vv, vh, path):
    # Create output directory if it doesn't exist
    if not os.path.exists(path):
        os.makedirs(path)
    
    # Define spatial reference and geotransform
    crs = rio.crs.CRS.from_epsg(4326)
    transform = Affine.identity()
    
    # Save the processed VV and VH images as .tif files
    with rio.open(os.path.join(path, "vv_processed.tif"), "w", driver="GTiff", height=vv.shape[0], width=vv.shape[1], count=1, dtype=str(vv.dtype), crs=crs, transform=transform, compress="deflate", interleave="pixel") as vv_ds:
        vv_ds.write(vv, 1)
        
    with rio.open(os.path.join(path, "vh_processed.tif"), "w", driver="GTiff", height=vh.shape[0], width=vh.shape[1], count=1, dtype=str(vh.dtype), crs=crs, transform=transform, compress="deflate", interleave="pixel") as vh_ds:
        vh_ds.write(vh, 1)

# **Before Flood**

In [ ]:
#directory
file_path = '/content/drive/My Drive/satellite_data/keralabefore/'
vv_path = 'S1A_IW_GRDH_1SDV_20180610T004040_20180610T004105_022287_02697B_B830.SAFE/measurement/s1a-iw-grd-vv-20180610t004040-20180610t004105-022287-02697b-001.tiff'
vh_path = 'S1A_IW_GRDH_1SDV_20180610T004040_20180610T004105_022287_02697B_B830.SAFE/measurement/s1a-iw-grd-vh-20180610t004040-20180610t004105-022287-02697b-002.tiff'

In [ ]:
# Read in data
vv, vh = read_data(file_path,vv_path,vh_path)

In [ ]:
# Apply radiometric calibration
# vv_cal, vh_cal = apply_calibration(vv, vh)
vv_cal, vh_cal = apply_calibration2(file_path,vv_path,vh_path)

In [ ]:
#Visualisation
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6))
ax1.imshow(vv_cal, cmap='gray')
ax1.set_title('Calibrated VV')
ax1.axis('off')
ax2.imshow(vh_cal, cmap='gray')
ax2.set_title('Calibrated VH')
ax2.axis('off')
plt.show()

In [ ]:
# # vv_t,vh_t = apply_terrain_correction(vv_cal,vh_cal)
# vv_t,vh_t = apply_terrain_correction2(vv_cal,vh_cal)

In [ ]:
# fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6))
# ax1.imshow(vv_t, cmap='gray')
# ax1.set_title('Corrected Terrain VV')
# ax1.axis('off')
# ax2.imshow(vh_t, cmap='gray')
# ax2.set_title('Corrected Terrain VH')
# ax2.axis('off')
# plt.show()

In [ ]:
#Apply spekle filter
vv_filtered,vh_filtered = apply_speckle_filter(vv_cal,vh_cal)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6))
ax1.imshow(vv_filtered, cmap='gray')
ax1.set_title('Spekle filtered VV')
ax1.axis('off')
ax2.imshow(vh_filtered, cmap='gray')
ax2.set_title('Spekle filtered VH')
ax2.axis('off')
plt.show()

In [ ]:
save_bef = f'{file_path}/processed'

In [ ]:
# Save filtered images
save_images(vv_filtered, vh_filtered, save_bef)

# **During Flood**

In [ ]:
#directory
file_path1 = '/content/drive/My Drive/satellite_data/keraladuring/'
vv_path1 = 'S1A_IW_GRDH_1SDV_20180728T004043_20180728T004108_022987_027EB1_EA20.SAFE/measurement/s1a-iw-grd-vv-20180728t004043-20180728t004108-022987-027eb1-001.tiff'
vh_path1 = 'S1A_IW_GRDH_1SDV_20180728T004043_20180728T004108_022987_027EB1_EA20.SAFE/measurement/s1a-iw-grd-vh-20180728t004043-20180728t004108-022987-027eb1-002.tiff'

In [ ]:
# Apply radiometric calibration
vv_cal1, vh_cal1 = apply_calibration2(file_path1,vv_path1,vh_path1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6))
ax1.imshow(vv_cal1, cmap='gray')
ax1.set_title('Calibrated VV')
ax1.axis('off')
ax2.imshow(vh_cal1, cmap='gray')
ax2.set_title('Calibrated VH')
ax2.axis('off')
plt.show()

In [ ]:
#Apply speckle filter
vv_filtered1,vh_filtered1 = apply_speckle_filter(vv_cal1,vh_cal1)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6))
ax1.imshow(vv_filtered1, cmap='gray')
ax1.set_title('Spekle filtered VV')
ax1.axis('off')
ax2.imshow(vh_filtered1, cmap='gray')
ax2.set_title('Spekle filtered VH')
ax2.axis('off')
plt.show()

In [ ]:
save_dur = f'{file_path1}/processed'

In [ ]:
#saving the outputs
save_images(vv_filtered1, vh_filtered1, save_dur)

# **After Flood**

In [ ]:
#directory
file_path2 = '/content/drive/My Drive/satellite_data/keralaafter/'
vv_path2 = 'S1A_IW_GRDH_1SDV_20181020T004046_20181020T004111_024212_02A61E_2CDE.SAFE/measurement/s1a-iw-grd-vv-20181020t004046-20181020t004111-024212-02a61e-001.tiff'
vh_path2 = 'S1A_IW_GRDH_1SDV_20181020T004046_20181020T004111_024212_02A61E_2CDE.SAFE/measurement/s1a-iw-grd-vh-20181020t004046-20181020t004111-024212-02a61e-002.tiff'

In [ ]:
# Apply radiometric calibration
vv_cal2, vh_cal2 = apply_calibration2(file_path2,vv_path2,vh_path2)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6))
ax1.imshow(vv_cal2, cmap='gray')
ax1.set_title('Calibrated VV')
ax1.axis('off')
ax2.imshow(vh_cal2, cmap='gray')
ax2.set_title('Calibrated VH')
ax2.axis('off')
plt.show()

In [ ]:
#Apply speckle filter
vv_filtered2,vh_filtered2 = apply_speckle_filter(vv_cal2,vh_cal2)

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10, 6))
ax1.imshow(vv_filtered2, cmap='gray')
ax1.set_title('Spekle filtered VV')
ax1.axis('off')
ax2.imshow(vh_filtered2, cmap='gray')
ax2.set_title('Spekle filtered VH')
ax2.axis('off')
plt.show()

In [ ]:
save_aft = f'{file_path2}/processed'

In [ ]:
#saving the outputs
save_images(vv_filtered2, vh_filtered2, save_aft)